This notebook contains implementations (not library usages) of the algorithms from Numerical Linear Algebra lectures.

*Slicing is used when available, leaving for loops commented.

In [339]:
import numpy as np

## $LU$ GE No P

In [340]:
def lunpiv(A_,ptol=10**-10): #return L,U. No pivoting.
    A = A_.copy()
    n=A.shape[0]
    for k in range(n-1):
        pivot = A[k,k]
        if abs(pivot)<ptol:
            print('zero pivot encountered')
            break
        A[k+1:,k]/=pivot
        A[k+1:,k+1:]-=np.outer(A[k+1:,k],A[k,k+1:])
        #for i in range(k+1,n):
        #    A[i,k]=A[i,k]/pivot
        #    for j in range(k+1,n):
        #        A[i,j]=A[i,j]-A[i,k]*A[k,j]
        #A is used to store both L and U. Next we split them
    L=np.eye(n)+np.tril(A,-1)
    U=np.triu(A)
    return L,U

## $LA=B$

In [341]:
def L_solve(L,B):
    """Find A so that L*A = B profiting the lower triangular structure of L."""
    n = L.shape[1]#number of rows of A = number of cols of L
    m = B.shape[1]#number of cols of A = number of cols of B
    A = np.zeros((n,m))
    for i in range(n):
        A[i,:]=(B[i,:]-L[i,:i].dot(A[:i,:]))/L[i,i]
        #for j in range(m):
        #    A[i,j]=B[i,j]
        #    for k in range(i):
        #        A[i,j]-=L[i,k]*A[k,j]
        #    A[i,j]/=L[i,i]
    return A

n=2
B = np.random.rand(n,n)
print("B\n", B)
L,U = lunpiv(B)
print("L\n", L)
print("U\n", U)
U_s = L_solve(L,B)
print("U_s\n", U_s)
'err',np.max(abs(U_s-U))

B
 [[ 0.94762644  0.23748577]
 [ 0.22951114  0.63992596]]
L
 [[ 1.         0.       ]
 [ 0.2421958  1.       ]]
U
 [[ 0.94762644  0.23748577]
 [ 0.          0.58240791]]
U_s
 [[ 0.94762644  0.23748577]
 [ 0.          0.58240791]]


('err', 0.0)

In [342]:
def U_solve(U,B):
    """Find A so that U*A = B profiting the upper triangular structure of U."""
    n = U.shape[1]#number of rows of A = number of cols of L
    m = B.shape[1]#number of cols of A = number of cols of B
    A = np.zeros((n,m))
    for i in range(n-1,-1,-1):
        A[i,:]=(B[i,:]-U[i,i+1:].dot(A[i+1:,:]))/U[i,i]
    return A

B=np.random.rand(n,n)
A=U_solve(U,B)
'err',np.max(abs(U.dot(A)-B))

('err', 1.1102230246251565e-16)

## $LL^T$ Cholesky

In [343]:
def Cholesky(A):
    n=A.shape[0]
    L=np.zeros((n,n))
    for j in range(n):
        L[j,j]=np.sqrt( A[j,j]-L[j,:j].dot(L[j,:j]) )
        #L[j,j]=A[j,j]
        #for k in range(j):
        #    L[j,j]-=L[j,k]*L[j,k]
        #L[j,j]=np.sqrt(L[j,j])
        for i in range(j+1,n):
            L[i,j]=( A[i,j]-L[i,:j].dot(L[j,:j]) )/L[j,j]
            #L[i,j]=A[i,j]
            #for k in range(j):
            #    L[i,j]-=L[i,k]*L[j,k]
            #L[i,j]/=L[j,j]
    return L
A=np.array([
[25, 15, -5],
[15, 18,  0],
[-5,  0,  11]])
L = Cholesky(A)
print('L\n',L)
print('max abs err',np.max(abs(L.dot(L.T)-A)))

L
 [[ 5.  0.  0.]
 [ 3.  3.  0.]
 [-1.  1.  3.]]
max abs err 0.0


## LS using cholesky

In [344]:
def LS_cholesky(A,b):
    """Solve A^TAx=A^Tb from Ax=b"""
    L=Cholesky(A.T.dot(A))#LLT = ATA
    LTx = L_solve(L,A.T.dot(b))
    x = U_solve(L.T,LTx)
    return x

A=np.array([
    [1,2],
    [3,4],
    [5,6],
])
b=np.array([
    [0],
    [1],
    [-1]
])
x=LS_cholesky(A,b)
print('x',x)
print('err',np.linalg.norm(A.dot(x)-b))

x [[-1.  ]
 [ 0.75]]
err 1.22474487139


## QR

### QR using Gram-Schmidt

In [345]:
def QR_gs(A):
    m,n=A.shape
    Q=np.zeros((m,n))
    R=np.zeros((n,n))
    for j in range(n):
        Q[:,j]=A[:,j]
        for i in range(j):
            R[i,j]=Q[:,i].T.dot(Q[:,j])
            Q[:,j]-=R[i,j]*Q[:,i]
        R[j,j]=np.linalg.norm(Q[:,j])
        Q[:,j]/=R[j,j]
    return Q,R

def QR_gs(A):
    m,n=A.shape
    Q=np.zeros((m,n))
    R=np.zeros((n,n))
    for i in range(n):
        Q[:,i]=A[:,i]
        for j in range(i):
            R[j,i]=Q[:,j].T.dot(Q[:,i])
            Q[:,i]-=R[j,i]*Q[:,j]
        R[i,i]=np.linalg.norm(Q[:,i])
        Q[:,i]/=R[i,i]
    return Q,R

print('A\n',A)
Q,R=QR_gs(A)
print('Q\n',Q)
print('R\n',R)
print('err',np.max(abs(A-Q.dot(R))))

A
 [[1 2]
 [3 4]
 [5 6]]
Q
 [[ 0.16903085  0.89708523]
 [ 0.50709255  0.27602622]
 [ 0.84515425 -0.34503278]]
R
 [[ 5.91607978  7.43735744]
 [ 0.          0.82807867]]
err 0.0


## LS using QR

In [346]:
def LS_qr(A,b):
    """Solve Ax=b with A=QR, Rx=Q^Tb"""
    Q,R = QR_gs(A)
    x = U_solve(R,Q.T.dot(b))
    return x

A=np.array([
    [1,2],
    [3,4],
    [5,6],
])
b=np.array([
    [0],
    [1],
    [-1]
])
x=LS_qr(A,b)
print('x',x)
print('err',np.linalg.norm(A.dot(x)-b))

x [[-1.  ]
 [ 0.75]]
err 1.22474487139


### QR using House-Holder

In [353]:
def QR_hh(A):
    m,n=A.shape
    Q=np.eye(m)
    R=A.copy()
    def House_P(x):
        print('x',x)
        u=x.copy()
        s=np.sign(x[0])
        if s==0:
            s=1
        u[0]+=s*np.linalg.norm(x)
        print('u',u)
        return np.eye(u.shape[0])-2*np.outer(u,u)/np.dot(u.T,u)
    for i in range(n):
        P=House_P(R[i:,i])
        print('i',i,'n',n,'m',m,'P shape',P.shape)
        if P.shape[0]!=m:
            P=np.bmat([[np.eye(i),np.zeros((i,m-i))],
                       [np.zeros((m-i,i)),P]])
        print('P',P)
        Q=Q.dot(P)
        R=P.dot(R)
        print('Q\n',Q)
        print('R\n',R)
        print('QTA is\n',Q.T.dot(A))
    return Q[:,:n],R[:n,:n]

B=np.array([
    [1,-1,4],
    [1,4,-2],
    [1,4,2],
    [1,-1,0]
])
print('A\n',B)
Q,R=QR_hh(B)
print('Q\n',Q)
print('R\n',R)
print('err',np.max(abs(B-Q.dot(R))))

A
 [[ 1 -1  4]
 [ 1  4 -2]
 [ 1  4  2]
 [ 1 -1  0]]
x [1 1 1 1]
u [3 1 1 1]
i 0 n 3 m 4 P shape (4, 4)
P [[-0.5        -0.5        -0.5        -0.5       ]
 [-0.5         0.83333333 -0.16666667 -0.16666667]
 [-0.5        -0.16666667  0.83333333 -0.16666667]
 [-0.5        -0.16666667 -0.16666667  0.83333333]]
Q
 [[-0.5        -0.5        -0.5        -0.5       ]
 [-0.5         0.83333333 -0.16666667 -0.16666667]
 [-0.5        -0.16666667  0.83333333 -0.16666667]
 [-0.5        -0.16666667 -0.16666667  0.83333333]]
R
 [[ -2.00000000e+00  -3.00000000e+00  -2.00000000e+00]
 [  1.11022302e-16   3.33333333e+00  -4.00000000e+00]
 [  5.55111512e-17   3.33333333e+00   5.55111512e-17]
 [  1.11022302e-16  -1.66666667e+00  -2.00000000e+00]]
QTA is
 [[ -2.00000000e+00  -3.00000000e+00  -2.00000000e+00]
 [  1.11022302e-16   3.33333333e+00  -4.00000000e+00]
 [  5.55111512e-17   3.33333333e+00   5.55111512e-17]
 [  1.11022302e-16  -1.66666667e+00  -2.00000000e+00]]
x [ 3.33333333  3.33333333 -1.6666666

### QR using Givens Rotation

In [354]:
def QR_gr(A):
    m,n=A.shape
    Q=np.eye(m)
    R=A.copy()
    def Givens(xi,xj,i,j,m):
        mu=np.sqrt(xi**2+xj**2)
        cos=xi/mu
        sin=xj/mu
        P=np.eye(m)
        P[i,i]=cos
        P[i,j]=sin
        P[j,i]=-sin
        P[j,j]=cos
        return P
    for j in range(n):
        for i in range(j+1,m):
            print('\ncancel entry',i+1,j+1)
            P=Givens(R[j,j],R[i,j],j,i,m)
            print('P',P)
            Q=Q.dot(P.T)
            R=P.dot(R)
            print('Q\n',Q)
            print('R\n',R)
            print('QTA is\n',Q.T.dot(A))        
    return Q[:,:n],R[:n,:n]

B=np.array([
    [1,-1,4],
    [1,4,-2],
    [1,4,2],
    [1,-1,0]
])
print('A\n',B)
Q,R=QR_hh(B)
print('Q\n',Q)
print('R\n',R)
print('err',np.max(abs(B-Q.dot(R))))

A
 [[ 1 -1  4]
 [ 1  4 -2]
 [ 1  4  2]
 [ 1 -1  0]]
x [1 1 1 1]
u [3 1 1 1]
i 0 n 3 m 4 P shape (4, 4)
P [[-0.5        -0.5        -0.5        -0.5       ]
 [-0.5         0.83333333 -0.16666667 -0.16666667]
 [-0.5        -0.16666667  0.83333333 -0.16666667]
 [-0.5        -0.16666667 -0.16666667  0.83333333]]
Q
 [[-0.5        -0.5        -0.5        -0.5       ]
 [-0.5         0.83333333 -0.16666667 -0.16666667]
 [-0.5        -0.16666667  0.83333333 -0.16666667]
 [-0.5        -0.16666667 -0.16666667  0.83333333]]
R
 [[ -2.00000000e+00  -3.00000000e+00  -2.00000000e+00]
 [  1.11022302e-16   3.33333333e+00  -4.00000000e+00]
 [  5.55111512e-17   3.33333333e+00   5.55111512e-17]
 [  1.11022302e-16  -1.66666667e+00  -2.00000000e+00]]
QTA is
 [[ -2.00000000e+00  -3.00000000e+00  -2.00000000e+00]
 [  1.11022302e-16   3.33333333e+00  -4.00000000e+00]
 [  5.55111512e-17   3.33333333e+00   5.55111512e-17]
 [  1.11022302e-16  -1.66666667e+00  -2.00000000e+00]]
x [ 3.33333333  3.33333333 -1.6666666

## SVD $A=U\Sigma V^T$

In [392]:
from scipy import linalg as spla
def SVD_spec(A):
    m,n=A.shape
    AtA=np.dot(A.T,A)
    AAt=np.dot(A,A.T)
        
    vaps_AAt, veps_AAt = spla.eig(AAt,right=True)
    vaps_AtA, veps_AtA = spla.eig(AtA,right=True)
    print("eigenvalues AAt\n", vaps_AAt)
    print("eigenvalues AtA\n", vaps_AtA)
    L_AAt = np.argsort(vaps_AAt)[::-1]
    L_AtA = np.argsort(vaps_AtA)[::-1]
    S = np.zeros((m,n))
    for i in range(0,n):
        S[i,i] = np.sqrt(abs(vaps_AAt[L_AAt[i]]))
    U = np.zeros((m,m))
    for i in range(0,m):
        U[i] = veps_AAt[L_AAt[i]]
    Vt = np.zeros((n,n))
    for i in range(0,n):
        Vt[i] = veps_AtA[L_AtA[i]]
    print("U=\n", U)
    print("V^T=\n", Vt)
    print("S=\n", S)
    print('USVt (=A up to column change of sign)\n',U.dot(S).dot(Vt))

print('A',A)
SVD_spec(A)

A [[1 2]
 [3 4]
 [5 6]]
eigenvalues AAt
 [  9.07354949e+01+0.j   2.64505087e-01+0.j  -3.00460789e-15+0.j]
eigenvalues AtA
 [  0.26450509+0.j  90.73549491+0.j]
U=
 [[-0.2298477  -0.88346102  0.40824829]
 [-0.52474482 -0.24078249 -0.81649658]
 [-0.81964194  0.40189603  0.40824829]]
V^T=
 [[ 0.61962948 -0.78489445]
 [-0.78489445 -0.61962948]]
S=
 [[ 9.52551809  0.        ]
 [ 0.          0.51430058]
 [ 0.          0.        ]]
USVt (=A up to column change of sign)
 [[-1.  2.]
 [-3.  4.]
 [-5.  6.]]


This implementation is inefficient and just for showing, but we can relate its result to LS

### LS using SVD

In [394]:
def pseudo_inv(A):
    m,n=A.shape
    U,s,V=np.linalg.svd(A)
    print('U\n',U)
    print('s',s)
    print('V',V)
    S_inv = np.zeros((m,n))
    for i in range(n):
        S_inv[i,i]=1/s[i]
    return V.dot(S_inv.T).dot(U.T)

def LS_svd(A,b):    
    pseudo_inv_A = pseudo_inv(A)
    return pseudo_inv_A.dot(b)

A=np.array([
    [1,2],
    [3,4],
    [5,6],
])
b=np.array([
    [0],
    [1],
    [-1]
])
x=LS_svd(A,b)
print('x',x)
print('err',np.linalg.norm(A.dot(x)-b))

U
 [[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
s [ 9.52551809  0.51430058]
V [[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]
x [[-1.  ]
 [ 0.75]]
err 1.22474487139


## Power method

In [527]:
def power_method(A):
    x=np.ones(A.shape[1])
    y=2*x
    k=1
    while np.linalg.norm(x-y,np.inf)>1e-10:
        x=y
        y=A.dot(x)
        y/=np.linalg.norm(y)
        k+=1
    return x

A=np.array([[1,2],[3,4]])
print('A',A)
x=power_method(A)
print('unit eigenvector',x)
print('Ax/x',A.dot(x)/x)
print('eigenvalue',x.dot(A).dot(x))

A [[1 2]
 [3 4]]
unit eigenvector [ 0.41597356  0.90937671]
Ax/x [ 5.37228132  5.37228132]
eigenvalue 5.37228132326


## Orthogonal Iteration (Schur)

In [530]:
def OI(A):
    Q=np.eye(A.shape[0])
    R=np.eye(A.shape[0])
    Y=A.dot(Q)
    X=Y.copy()
    k=1
    while k<3 or spla.norm(Y-X,np.inf)>1e-10:
        X=Y.copy()      
        Y=A.dot(Q)
        Q,R=np.linalg.qr(Y)
        k+=1
    print(k,'iterations')
    return Q,R

print('A',A)
Q,R=OI(A)
print('R (schur form)\n',R)
print('Q\n',Q)
print('QRQt recovers A\n',Q.dot(R).dot(Q.T))

A [[1 2]
 [3 4]]
13 iterations
R (schur form)
 [[ 5.37228132  1.        ]
 [ 0.         -0.37228132]]
Q
 [[-0.41597356 -0.90937671]
 [-0.90937671  0.41597356]]
QRQt recovers A
 [[ 1.  2.]
 [ 3.  4.]]


## QR iteration

In [472]:
def QRI(A):
    Y=A
    Q,R=np.zeros(n),np.zeros(n)
    X=Y.copy()
    k=1
    while k<3 or spla.norm(Y-X,np.inf)>1e-10:
        X=Y.copy()
        Q,R=np.linalg.qr(Y)
        Y=R.dot(Q)
        k+=1
    print(k,'iterations')
    return Q,R

print('A',A)
Q,R=QRI(A)
print('R\n',R)
print('Q\n',Q)
print('QRQt is R\n',Q.dot(R).dot(Q.T))

A [[1 2]
 [3 4]]
283 iterations
R
 [[ 5.37228132  1.        ]
 [ 0.         -0.37228132]]
Q
 [[ 1.  0.]
 [-0.  1.]]
QRQt is R
 [[ 5.37228132  1.        ]
 [ 0.         -0.37228132]]


Actually this is for hessenberg...

## Iterative Methods

In [532]:
A=np.array(
[
    [4,-1,-1,0],
    [-1,4,0,-1],
    [-1,0,4,-1],
    [0,-1,-1,4]
])
b=np.array([1,2,0,1])
n=A.shape[0]

def LDU(A):
    L=np.tril(A,-1)
    D=np.diag(np.diag(A))
    U=np.triu(A,1)
    return L,D,U
L,D,U=LDU(A)
#print(L,'\n',D,'\n',U)

### Jacobi

In [533]:
def jacobistep(A,b,x):
    #n=A.shape[1]
    #y=np.zeros(n)
    #for i in range(n):
    #    suma=0
    #    for j in range(i):
    #        suma+=A[i,j]*x[j]
    #    for j in range(i+1,n):
    #        suma+=A[i,j]*x[j]
    #    y[i]=(b[i]-suma)/A[i,i]
    #return y
    ##equivalent to 
    return x+(b-np.dot(A,x))/np.diag(A)
    
def Jacobi(A,b,x_0):
    nor = 1
    k=0
    while nor > 1e-9 and k<100:
        x=jacobistep(A,b,x_0)
        k = k+1
        nor = np.linalg.norm(x-x_0)
        x_0 = x
    print(k,'iterations')
    return x

x_0=np.zeros(n)
x = Jacobi(A,b,x_0)
print('x',x)
print('|Ax-b|',np.linalg.norm(A.dot(x)-b))

R_J = np.dot(np.linalg.inv(D),L+U)
rho = max(abs(np.linalg.eigvals(R_J)))
print('Spectral Radius RJ =', rho)
print('Rate of convergence RJ = ', -np.log10(rho))

30 iterations
x [ 0.5   0.75  0.25  0.5 ]
|Ax-b| 1.86264514923e-09
Spectral Radius RJ = 0.5
Rate of convergence RJ =  0.301029995664


### Gauss Seidel

In [534]:
def gauss_seidelstep(A,b,x):
    n=A.shape[1]
    y=np.zeros(n)
    for i in range(n):
        suma=0
        for j in range(i):
            suma+=A[i,j]*y[j]
        for j in range(i+1,n):
            suma+=A[i,j]*x[j]
        y[i]=(b[i]-suma)/A[i,i]
    return y

def Gauss_Seidel(A,b,x_0):
    nor = 1
    k=0
    while nor > 1e-9 and k<100:
        x = gauss_seidelstep(A,b,x_0)
        k = k+1
        nor = np.linalg.norm(x-x_0)
        x_0 = x
    print(k,'iterations')
    return x

x_0=np.zeros(n)
x = Gauss_Seidel(A,b,x_0)
print('x',x)
print('|Ax-b|',np.linalg.norm(A.dot(x)-b))

R_GS = np.dot(np.linalg.inv(D-L),U)
rho = max(abs(np.linalg.eigvals(R_GS)))
print('Spectral Radius RGS =', rho)
print('Rate of convergence RGS = ', -np.log10(rho))

17 iterations
x [ 0.5   0.75  0.25  0.5 ]
|Ax-b| 2.77823464305e-10
Spectral Radius RGS = 0.25
Rate of convergence RGS =  0.602059991328


### SOR(w)

In [524]:
def SORstep(A,b,x,w):
    n=A.shape[1]
    y=np.zeros(n)
    for i in range(0,n):
        suma=0
        for j in range(0,i):
            suma=suma+A[i,j]*y[j]
        for j in range(i+1,n):
            suma=suma+A[i,j]*x[j]
        y[i] = (1-w)*x[i]+w*(b[i]-suma)/A[i,i]
    #y = (1-w)*x + w*gauss_seidelstep(A,b,x) #not the same, recursion on i, not parallelism
    return y

def SOR(A,b,x_0,w):
    nor = 1
    k=0
    while nor > 1e-9 and k<100:
        x = SORstep(A,b,x_0,w)
        k = k+1
        nor = np.linalg.norm(x-x_0)
        x_0 = x
    print(k,'iterations')
    return x

w = 0.5
x_0=np.zeros(n)
x = SOR(A,b,x_0,w)
print('x',x)
print('|Ax-b|',np.linalg.norm(A.dot(x)-b))

R_SOR = np.dot(np.linalg.inv(D-w*L),((1-w)*D + w*U))
rho = max(abs(np.linalg.eigvals(R_SOR)))
print('Spectral Radius RGS =', rho)
print('Rate of convergence RGS = ', -np.log10(rho))

59 iterations
x [ 0.5   0.75  0.25  0.5 ]
|Ax-b| 3.78450073651e-09
Spectral Radius RGS = 0.710767582704
Rate of convergence RGS =  0.148272388089
